In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

# -------------------------
# Load All Data
# -------------------------
train = pd.read_csv("train.csv", parse_dates=["date"])
test = pd.read_csv("test.csv", parse_dates=["date"])
holidays = pd.read_csv("holidays_events.csv", parse_dates=["date"])
stores = pd.read_csv("stores.csv")
oil = pd.read_csv("oil.csv", parse_dates=["date"])
sample_submission = pd.read_csv("sample_submission.csv")

# -------------------------
# Preprocess Holidays
# -------------------------
def merge_holidays(df, holidays):
    holidays_filtered = holidays[holidays["transferred"] == False]
    holidays_filtered = holidays_filtered[["date", "description"]].drop_duplicates()
    holidays_filtered["is_holiday"] = 1
    return df.merge(holidays_filtered, on="date", how="left").fillna({"is_holiday": 0})

train = merge_holidays(train, holidays)
test = merge_holidays(test, holidays)

# -------------------------
# Create Date Features
# -------------------------
for df in [train, test]:
    df["day"] = df["date"].dt.day
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["dayofweek"] = df["date"].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)

# -------------------------
# Encode Categorical Features
# -------------------------
le_store = LabelEncoder()
le_family = LabelEncoder()
le_city = LabelEncoder()
le_state = LabelEncoder()
le_type = LabelEncoder()

train["store_nbr_enc"] = le_store.fit_transform(train["store_nbr"])
test["store_nbr_enc"] = le_store.transform(test["store_nbr"])

train["family_enc"] = le_family.fit_transform(train["family"])
test["family_enc"] = le_family.transform(test["family"])

# -------------------------
# Merge Store Info
# -------------------------
stores["city_enc"] = le_city.fit_transform(stores["city"])
stores["state_enc"] = le_state.fit_transform(stores["state"])
stores["type_enc"] = le_type.fit_transform(stores["type"])

train = train.merge(stores, on="store_nbr", how="left")
test = test.merge(stores, on="store_nbr", how="left")

# -------------------------
# Process Oil Prices
# -------------------------
oil["dcoilwtico"] = oil["dcoilwtico"].ffill().bfill()
oil["oil_7d_avg"] = oil["dcoilwtico"].rolling(window=7, min_periods=1).mean()
oil["oil_30d_avg"] = oil["dcoilwtico"].rolling(window=30, min_periods=1).mean()

train = train.merge(oil, on="date", how="left")
test = test.merge(oil, on="date", how="left")

# -------------------------
# Final Feature Set
# -------------------------
features = [
    "store_nbr_enc", "family_enc", "onpromotion", "day", "month", "year",
    "dayofweek", "is_weekend", "is_holiday", "city_enc", "state_enc", "type_enc",
    "cluster", "dcoilwtico", "oil_7d_avg", "oil_30d_avg"
]

# Drop rows with missing data
train = train.dropna(subset=features)

# -------------------------
# Train/Validation Split
# -------------------------
X = train[features]
y = train["sales"]
X_test = test[features]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# -------------------------
# Train Model
# -------------------------
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# -------------------------
# Validate
# -------------------------
val_preds = np.maximum(0, model.predict(X_val))
rmsle = np.sqrt(mean_squared_log_error(y_val + 1, val_preds + 1))  # more robust
print("Validation RMSLE:", rmsle)

# -------------------------
# Predict on Test Set
# -------------------------
test_preds = np.maximum(0, model.predict(X_test))
test["sales"] = test_preds
submission = test[["id", "sales"]]
submission.to_csv("submission.csv", index=False)

# -------------------------
# Save Cleaned Data for Power BI
# -------------------------
train["source"] = "train"
test["source"] = "test"
test["sales"] = test_preds
clean_data = pd.concat([train, test], sort=False).reset_index(drop=True)
clean_data.to_csv("clean_data.csv", index=False)

print("Files saved: 'submission.csv' and 'clean_data.csv'.")


Validation RMSLE: 0.4086864483537214
Files saved: 'submission.csv' and 'clean_data.csv'.
